In [2]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from skimage.transform import resize
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix
from glob import glob
import os
from itertools import chain
from PIL import Image
from keras.preprocessing import image

In [33]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('\n ********* Load file {} *********'.format(filename))
    ds = pydicom.dcmread(filename)       
    img = ds.pixel_array
    # dicom checks
    print('Patient ID: ', ds.PatientID)
    print('Patient Postion: ', ds.PatientPosition)
    print('Patient Age: ', ds.PatientAge)
    print('Patient Sex: ', ds.PatientSex)
    print('Body Part Examined: ', ds.BodyPartExamined)
    print('Photometric Interpretation: ', ds.PhotometricInterpretation)
    print('Study Description: ', ds.StudyDescription)
    print('Image type: ', ds.Modality)
    
    if ds.BodyPartExamined != 'CHEST':
        print('--------------------------------------------')
        print(" Invalid image, prediction won't be reliable\n due to being different from a Chest body part")
        print('--------------------------------------------')
    if ds.PatientPosition != 'PA' and ds.PatientPosition != 'AP':
        print('--------------------------------------------')
        print(" Invalid image, prediction won't be reliable\n due to having an invalid Image position")
        print('--------------------------------------------')
    if ds.Modality != 'DX':
        print('--------------------------------------------')
        print(" Invalid image, prediction won't be reliable\n due to having an invalid Image type")
        print('--------------------------------------------')
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    ## images resize by "resize" already come in a 0 to 1 range in case
    #they aren't. No need to normalize diving by 255.
    resized_img = resize(img, (img_size, img_size), anti_aliasing=True)
    #img = data_gen.standardize(img)
    
    ## Since no standarization was done during training we will leave
    # The next line as a place holder in case we perform standarization
    # in the future
    proc_img = (img - img_mean) / img_std 
    
    proc_img = np.zeros((224,224,3))
    proc_img[:, :, 0] = resized_img
    proc_img[:, :, 1] = resized_img
    proc_img[:, :, 2] = resized_img
    proc_img = np.expand_dims(proc_img, axis=0)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_model = open(model_path, 'r')
    loaded_model_json = json_model.read()
    json_model.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    return model
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh, numericLabels=False): 
    # todo    
    result = model.predict(img)  
    predict=result[0]
    if numericLabels:
        prediction = 0
        if(predict > thresh):
            prediction = 1
    else:
        prediction = "Prediction: Negative"
        if(predict > thresh):
            prediction = "Prediction: Positive"
    return prediction 

def read_img(path_to_img_file):
    #image = np.array(Image.open(path_to_img_file))
    img = image.load_img(path_to_img_file, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = img/255.0
    return img

In [34]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json' #path to saved model
weight_path = 'xray_class_my_model.best.hdf5' #path to saved best weights

IMG_SIZE=224 # This might be different if you did not use vgg16
img_mean = 0# loads the mean image value they used during training preprocessing
img_std = 1# loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)#loads model
thresh = 0.6507165431976318 #loads the threshold they chose for model classification 
# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)


 ********* Load file test1.dcm *********
Patient ID:  2
Patient Postion:  PA
Patient Age:  81
Patient Sex:  M
Body Part Examined:  CHEST
Photometric Interpretation:  MONOCHROME2
Study Description:  No Finding
Image type:  DX
Prediction: Negative

 ********* Load file test2.dcm *********
Patient ID:  1
Patient Postion:  AP
Patient Age:  58
Patient Sex:  M
Body Part Examined:  CHEST
Photometric Interpretation:  MONOCHROME2
Study Description:  Cardiomegaly
Image type:  DX
Prediction: Negative

 ********* Load file test3.dcm *********
Patient ID:  61
Patient Postion:  AP
Patient Age:  77
Patient Sex:  M
Body Part Examined:  CHEST
Photometric Interpretation:  MONOCHROME2
Study Description:  Effusion
Image type:  DX
Prediction: Positive

 ********* Load file test4.dcm *********
Patient ID:  2
Patient Postion:  PA
Patient Age:  81
Patient Sex:  M
Body Part Examined:  RIBCAGE
Photometric Interpretation:  MONOCHROME2
Study Description:  No Finding
Image type:  DX
------------------------------

**Note:** Probably the reason the model classified Effusion as a positive case is because it is correlated to Pneumonia as was found in the EDA. Effusion is one of the top 5 comorbidities related to Pneumonia

### Investigate algorithm in presence of other disceases

In [8]:
all_xray_df = pd.read_csv('/data/Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('/data','images*', '*', '*.png'))}
print('Scans found:', len(all_image_paths), ', Total Headers', all_xray_df.shape[0])
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
all_labels = np.unique(list(chain(*all_xray_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
print(f'We have {len(all_labels)} unique labels and they are: {all_labels}')
i = 2
for label in all_labels:
    # lets create a column based on each disease and set to 1 if the patient has it.
    all_xray_df.insert(i, label, all_xray_df['Finding Labels'].map(lambda str_label: 1.0 if label in str_label else 0))
    i=i+1
pd.set_option('display.max_columns', None) #if set to None the dataframe displays all columns.
all_xray_df.sample(3)

Scans found: 112120 , Total Headers 112120
We have 15 unique labels and they are: ['Atelectasis' 'Cardiomegaly' 'Consolidation' 'Edema' 'Effusion'
 'Emphysema' 'Fibrosis' 'Hernia' 'Infiltration' 'Mass' 'No Finding'
 'Nodule' 'Pleural_Thickening' 'Pneumonia' 'Pneumothorax']


,Image Index,Finding Labels,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,path
48522,00012298_016.png,No Finding,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,16,12298,64,M,PA,2500,2048,0.168,0.168,NaN,/data/images_006/images/00012298_016.png
5693,00001535_000.png,No Finding,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,1535,62,F,PA,2500,2048,0.168,0.168,NaN,/data/images_002/images/00001535_000.png
107381,00028987_016.png,Infiltration,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,16,28987,25,F,AP,3056,2544,0.139,0.139,NaN,/data/images_012/images/00028987_016.png


In [204]:
## Data only for more related to pneumonia diseases based on EDA 
# DISEASES are Infiltration, Edema, Atelectasis
rel_indexes = all_xray_df['Finding Labels'].eq('Infiltration|Pneumonia') | \
                all_xray_df['Finding Labels'].eq('Edema|Infiltration|Pneumonia') | \
                all_xray_df['Finding Labels'].eq('Atelectasis|Pneumonia')
comorbidities_df = all_xray_df[rel_indexes]
Pneumonia_df = all_xray_df[all_xray_df['Finding Labels'].eq('Pneumonia')]

In [206]:
predictions=[]
true_values=[]
for path_to_img in all_xray_df[all_xray_df['No Finding']==1][['path']].sample(5).values.reshape(-1):
    print(path_to_img)
    img_proc = read_img(path_to_img)
    pred = predict_image(my_model,img_proc,thresh, numericLabels = True)
    #pred = my_model.predict(img_proc, batch_size = 100, verbose = True)
    predictions.append(pred)
    true_values.append(0)
random_samples = comorbidities_df.groupby('Finding Labels').apply(lambda comorbidities_df: comorbidities_df.sample(5))
for path_to_img in random_samples['path'].values:
    print(path_to_img)
    img_proc = read_img(path_to_img)
    pred = predict_image(my_model,img_proc,thresh, numericLabels = True)
    predictions.append(pred)
print(predictions)

/data/images_003/images/00005435_000.png
/data/images_001/images/00001054_002.png
/data/images_004/images/00007629_001.png
/data/images_001/images/00000773_001.png
/data/images_004/images/00007629_002.png
/data/images_009/images/00019785_019.png
/data/images_002/images/00003699_000.png
/data/images_009/images/00019018_003.png
/data/images_012/images/00029654_000.png
/data/images_005/images/00010780_007.png
/data/images_010/images/00022698_001.png
/data/images_010/images/00022021_002.png
/data/images_009/images/00019865_014.png
/data/images_007/images/00015275_007.png
/data/images_005/images/00010706_000.png
['Negative', 'Negative', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive']


In [234]:
predictions=[]
true_values=[]
for path_to_img in all_xray_df[all_xray_df['No Finding']==1][['path']].sample(5).values.reshape(-1):
    print(path_to_img)
    img_proc = read_img(path_to_img)
    pred = predict_image(my_model,img_proc,thresh, numericLabels = True)
    #pred = my_model.predict(img_proc, batch_size = 100, verbose = True)
    predictions.append(pred)
    true_values.append(0)
samples = Pneumonia_df[Pneumonia_df['Pneumonia']==1][['path']].sample(5)
for path_to_img in samples.values.reshape(-1):
    print(path_to_img)
    img_proc = read_img(path_to_img)
    pred = predict_image(my_model,img_proc,thresh, numericLabels = True)
    #pred = my_model.predict(img_proc, batch_size = 100, verbose = True)
    predictions.append(pred)
    true_values.append(1)
print(predictions)

/data/images_005/images/00010531_073.png
/data/images_008/images/00017254_000.png
/data/images_002/images/00003510_009.png
/data/images_005/images/00011553_045.png
/data/images_011/images/00026366_001.png
[0, 0, 0, 0, 0, 0, 1, 1, 1, 0]


In [239]:
print("********************* Only Pneumonia test *********************")
tn, fp, fn, tp = confusion_matrix(true_values, predictions,labels=[1,0]).ravel()
print(f'True positives: {tp}, True negatives: {tn}, False positives: {fp}, False negatives: {fn}')
sens = tp/(tp+fn)
spec = tn/(tn+fp)
prec = tp/(tp+fp)
print(f'sensibility: {sens}, specificity: {spec}, precision: {prec}')

********************* Only Pneumonia test *********************
True positives: 5, True negatives: 3, False positives: 2, False negatives: 0
sensibility: 1.0, specificity: 0.6, precision: 0.7142857142857143


In [281]:
number_of_samples = 10
dictPrediction={}
dictTrueVal={}
random_samples = comorbidities_df.groupby('Finding Labels').apply(
    lambda comorbidities_df: comorbidities_df.sample(number_of_samples))
for class_name in random_samples.index.levels[0]:
    predictions=[]
    true_values=[]
    print(f'Loading {class_name}')
    for path_to_img in all_xray_df[all_xray_df['No Finding']==1][['path']].sample(
                                                                number_of_samples).values.reshape(-1):
        img_proc = read_img(path_to_img)
        pred = predict_image(my_model,img_proc,thresh, numericLabels = True)
        #pred = my_model.predict(img_proc, batch_size = 100, verbose = True)
        predictions.append(pred)
        true_values.append(0)
    random_class = random_samples.loc[class_name]
    for path_to_img in random_class['path'].values:
        img_proc = read_img(path_to_img)
        pred = predict_image(my_model,img_proc,thresh, numericLabels = True)
        predictions.append(pred)
        true_values.append(1)
    dictPrediction[class_name]=predictions
    dictTrueVal[class_name]=true_values
print("********* Predictions dictionary *********")
print(dictPrediction)
print("********* True values dictionary *********")
print(dictTrueVal)

Loading Atelectasis|Pneumonia
Loading Edema|Infiltration|Pneumonia
Loading Infiltration|Pneumonia
********* Predictions dictionary *********
{'Atelectasis|Pneumonia': [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Edema|Infiltration|Pneumonia': [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0], 'Infiltration|Pneumonia': [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0]}
********* True values dictionary *********
{'Atelectasis|Pneumonia': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'Edema|Infiltration|Pneumonia': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'Infiltration|Pneumonia': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [282]:
for disease in dictPrediction.keys():
    print(f"\n********************* {disease} test *********************")
    tn, fp, fn, tp = confusion_matrix(dictTrueVal[disease], dictPrediction[disease],labels=[1,0]).ravel()
    print(f'True positives: {tp}, True negatives: {tn}, False positives: {fp}, False negatives: {fn}')
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    prec = tp/(tp+fp)
    print(f'sensibility: {sens}, specificity: {spec}, precision: {prec}')


********************* Atelectasis|Pneumonia test *********************
True positives: 8, True negatives: 0, False positives: 10, False negatives: 2
sensibility: 0.8, specificity: 0.0, precision: 0.4444444444444444

********************* Edema|Infiltration|Pneumonia test *********************
True positives: 8, True negatives: 7, False positives: 3, False negatives: 2
sensibility: 0.8, specificity: 0.7, precision: 0.7272727272727273

********************* Infiltration|Pneumonia test *********************
True positives: 8, True negatives: 6, False positives: 4, False negatives: 2
sensibility: 0.8, specificity: 0.6, precision: 0.6666666666666666


**Conclusion** Since I opted for prioritizing sensibility(recall) over precision and specificity I get higher values on recall on all tests for the threshold I selected. Because as said before for pneumonia detection, it is crucial that we find all the patients that have pneumonia. Predicting patients with pneumonia as healthy should not be acceptable. In other words, having high FN is not acceptable. This is measured by recall. 

However in the case of `Atelectasis` the algorithm performed very poorly. Hence this is cataloged as a limitation of this algorithm

In [288]:
!tar -czvf myProject.tar.gz .

./
./FDA_Submission_Template.md
./best_weights.hdf5
./Inference.ipynb
./test2.dcm
./history.csv
./test5.dcm
./test1.dcm
./test.npy
./test4.dcm
./Build and train model.ipynb
./test6.dcm
./EDA.ipynb
./test3.dcm
./sample_labels.csv
./FDA_Submission.md
./.ipynb_checkpoints/
./.ipynb_checkpoints/Build and train model-checkpoint.ipynb
./.ipynb_checkpoints/EDA-checkpoint.ipynb
./.ipynb_checkpoints/Inference-checkpoint.ipynb
./xray_class_my_model.best.hdf5
./my_model.json
tar: .: file changed as we read it
